### Deploy Result Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# data file parameter
file = "200 Classification"  # must be excel file
sheets = "Sheet3"  # "Sheet3"
start_first_column = 0  # start column location number

# classification for Part 1
threshold_num = 1

# word all usage in sent
word_use_num_max = 3
word_use_num_min = 1

In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [7]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [15]:
df_youtube_sent_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/\
6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_youtube_sent_result.sort_values(by="search_string", key=lambda x:x.str.len(), ascending=False, inplace=True)
df_youtube_sent_result.drop(["word_index"], axis=1, inplace=True)
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."
2,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
3,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...,"[107, 129, 2, 27, 36, 70, 7, 8, 94, 10]"
4,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
...,...,...,...,...,...,...,...
66,lütfen ilk önce onu,21,23,videoyu izlemediyseniz lütfen ilk önce onu izl...,DWIzzc0OaIM,https://www.youtube.com/watch?v=DWIzzc0OaIM&t=21s,"[58, 193, 51, 23]"
67,tam başka en uzun,361,366,o hep tam başka en uzun karbon zinciri simetrik,9MU9b0WWnNM,https://www.youtube.com/watch?v=9MU9b0WWnNM&t=...,"[112, 71, 44, 172]"
68,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
69,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"


In [16]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

TypeError: 'list' object is not callable

In [10]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index,word_index_list
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80...","[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,...","[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."
2,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]","[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
3,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...,"[107, 129, 2, 27, 36, 70, 7, 8, 94, 10]","[107, 129, 2, 27, 36, 70, 7, 8, 94, 10]"
4,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ...","[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
...,...,...,...,...,...,...,...,...
102,gerçekten çok güzel oldu gerçekten harika bir,373,378,yla döküyoruz bu şekilde kıvamları gerçekten ç...,oWLfHDt7KLg,https://www.youtube.com/watch?v=oWLfHDt7KLg&t=...,NaN,"[70, 9, 54, 55, 70, 115, 0]"
103,yani biraz bugün biraz üzgünüm neden,433,437,için çok semih yani biraz bugün biraz üzgünüm ...,DZmXkheI80A,https://www.youtube.com/watch?v=DZmXkheI80A&t=...,NaN,"[60, 52, 170, 52, 122, 34]"
104,söyle bakalım benimle şimdi geliyor musun,207,211,ee söyle bakalım benimle şimdi geliyor musun,9RP0Faoo2rw,https://www.youtube.com/watch?v=9RP0Faoo2rw&t=...,NaN,"[161, 175, 157, 43, 179, 53]"
105,çok teşekkür ederim ya ne demek ben teşekkür e...,649,651,çok tatlısın ömerciğim çok teşekkür ederim ya ...,9uEGE4v9ROM,https://www.youtube.com/watch?v=9uEGE4v9ROM&t=...,NaN,"[9, 126, 98, 45, 2, 86, 7, 126, 98]"


In [11]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [12]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [13]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[131,
 136,
 137,
 140,
 142,
 145,
 147,
 149,
 151,
 24,
 154,
 155,
 157,
 162,
 163,
 164,
 165,
 166,
 169,
 50,
 178,
 182,
 183,
 185,
 59,
 187,
 189,
 62,
 67,
 75,
 82,
 87,
 92,
 98,
 100,
 101,
 103,
 105,
 106,
 111,
 115,
 117,
 124,
 126,
 127]

In [14]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
1,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
2,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...,"[107, 129, 2, 27, 36, 70, 7, 8, 94, 10]"
3,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
4,aynı şekilde burada da var değil mi aynı şekil...,603,614,direnç konumda bakın aynı şekilde burada da va...,lkFe1n707zE,https://www.youtube.com/watch?v=lkFe1n707zE&t=...,"[130, 181, 38, 15, 11, 14, 5, 130, 181, 38, 11]"
5,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
6,bir şeyi ben böyle istiyorum ve bu şekilde ola...,16,21,herhangi bir şeyi ben böyle istiyorum ve bu şe...,CULrx3yerE8,https://www.youtube.com/watch?v=CULrx3yerE8&t=16s,"[0, 99, 7, 56, 65, 3, 1, 181, 83, 123]"
7,öyle olmak mı daha iyi böyle olmak mı daha bil...,431,434,öyle olmak mı daha iyi böyle olmak mı daha bil...,5B0XYZjzivw,https://www.youtube.com/watch?v=5B0XYZjzivw&t=...,"[46, 176, 13, 18, 28, 56, 176, 13, 18, 94]"
8,olabilir mi olabilir tabii ki belki de bu benim,408,411,farklılığı olabilir mi olabilir tabii ki belki...,5jZCJvqswAU,https://www.youtube.com/watch?v=5jZCJvqswAU&t=...,"[129, 5, 129, 128, 33, 74, 8, 1, 31]"
9,olsun güzel olsun efendim böyle senin olsun,679,684,benim seviyorum çok sıcağa demli olsun güzel o...,MKX0zLqY7YU,https://www.youtube.com/watch?v=MKX0zLqY7YU&t=...,"[116, 54, 116, 85, 56, 36, 116]"


In [ ]:
#df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx", index=False)